In [33]:
print("Hello! What is your name?")

Hello! What is your name?


In [36]:
name = input().upper()


bill nye the science pie


In [37]:
print("Hello, " +  name + "! How are you doing, " + name + "? My name is Bodie, " + name + ".")

Hello, BILL NYE THE SCIENCE PIE! How are you doing, BILL NYE THE SCIENCE PIE? My name is Bodie, BILL NYE THE SCIENCE PIE.


SyntaxError: invalid syntax (<ipython-input-32-25089fca76b1>, line 1)

In [23]:
##  This works with HTTPX
async def foo():
    async with httpx.AsyncClient() as client:
        url = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollectionValues'
        r = await client.get(url) #'https://www.example.com/')
    return r


for i in range(5):
    start = time.time()
    await asyncio.gather(*[foo() for x in range(i)])
    print(f'{i} runs in {time.time() - start} seconds')


NameError: name 'time' is not defined

# Call TCIA API

In [1]:
import httpx
import asyncio
import pandas as pd
import azure.storage.queue as asq
import os
import json
import time

# variables used throughout
tciabase = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query'


In [2]:
def handleRateLimits(headers):
    limits = dict([(h,int(headers[h])) for h in headers if h.find('x-rate') != -1])
    # Example : {'x-ratelimit-limit-hour': 360000, 'x-ratelimit-remaining-hour': 359961, 'x-ratelimit-limit-second': 1000, 'x-ratelimit-remaining-second': 999}
    
    # TODO: send a queue message to a queue which will 'pause' the querying from TCIA for the right time

## Get Collections

In [3]:
async def getCollectionsAsync():
    urlGetCollections = f'{tciabase}/getCollectionValues'
    async with httpx.AsyncClient() as client:
        r = await client.get(urlGetCollections)
    
    # not really needed here, but should send every time we send a request
    handleRateLimits(r.headers)
    
    if r is not None:
        #return [c['Collection'] for c in r.json()]
        return r.json()

In [12]:
#tcia_collections = await getCollectionsAsync()

In [14]:
#len(tcia_collections)
#print(tcia_collections)

[{'Collection': 'TCGA-GBM'}, {'Collection': 'LIDC-IDRI'}, {'Collection': 'BREAST-DIAGNOSIS'}, {'Collection': 'PROSTATE-MRI'}, {'Collection': 'PROSTATE-DIAGNOSIS'}, {'Collection': 'NaF PROSTATE'}, {'Collection': 'CT COLONOGRAPHY'}, {'Collection': 'REMBRANDT'}, {'Collection': 'RIDER Breast MRI'}, {'Collection': 'RIDER Lung CT'}, {'Collection': 'RIDER NEURO MRI'}, {'Collection': 'RIDER PHANTOM MRI'}, {'Collection': 'RIDER PHANTOM PET-CT'}, {'Collection': 'RIDER Lung PET-CT'}, {'Collection': 'QIBA CT-1C'}, {'Collection': 'Phantom FDA'}, {'Collection': 'TCGA-BRCA'}, {'Collection': 'QIN-BREAST'}, {'Collection': 'TCGA-LGG'}, {'Collection': 'TCGA-KIRC'}, {'Collection': 'TCGA-LUAD'}, {'Collection': 'Prostate-3T'}, {'Collection': 'QIN-HEADNECK'}, {'Collection': 'TCGA-PRAD'}, {'Collection': 'NSCLC Radiogenomics'}, {'Collection': 'Head-Neck Cetuximab'}, {'Collection': 'TCGA-LIHC'}, {'Collection': 'TCGA-KIRP'}, {'Collection': 'TCGA-OV'}, {'Collection': 'TCGA-HNSC'}, {'Collection': 'TCGA-KICH'}, {'C

In [7]:

# https://services.cancerimagingarchive.net/services/v3/TCIA/query/getPatientStudy?Collection=TCGA-GBM
# https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries?Collection=TCGA-GBM&StudyInstanceUID=1.3.6.1.4.1.14519.5.2.1.7695.4001.130563880911723253267280582465  
# https://services.cancerimagingarchive.net/services/v3/TCIA/query/getImage?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.7695.4001.306204232344341694648035234440



## Get PatientStudies in Collection

In [8]:
#------------------------------------------------------------------------------------------------------

SyntaxError: invalid syntax (<ipython-input-8-68a7df453531>, line 1)

In [9]:
#studies = await getPatientStudiesPerCollection(collections[0]['Collection'])

In [10]:
#print(len(studies))
#print(studies[0])

In [11]:
#study_ids = [s['StudyInstanceUID'] for s in studies] # if s.find('StudyInstanceUID') != -1]

## Get Series

In [11]:
async def getSeriesPerStudy(study, async_client):
    urlGetSeriesBase = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries'
    params = {'StudyInstanceUID': study}
    #async with httpx.AsyncClient() as client:
    r = await async_client.get(urlGetSeriesBase,params=params,timeout=None) #timeout=15.0)
    
    # not really needed here, but should send every time we send a request
    handleRateLimits(r.headers)
    
    if r is not None:
        #return [c['Collection'] for c in r.json()]
        return r.json()

In [12]:
#series = await getSeriesPerStudy(studies[0]['StudyInstanceUID'])

In [13]:
#len(series)
#print(series[0])

In [14]:
async def getSeriesPerStudy(study, async_client):
    urlGetSeriesBase = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries'
    params = {'StudyInstanceUID': study}
    #async with httpx.AsyncClient() as client:
    r = await async_client.get(urlGetSeriesBase,params=params,timeout=None) #timeout=15.0)
    
    # not really needed here, but should send every time we send a request
    handleRateLimits(r.headers)
    
    if r is not None:
        #return [c['Collection'] for c in r.json()]
        return r.json()

In [15]:
async def getSeriesPerStudyJoinDict(study_id, study_dict, async_client):
    
    #retry three times if necessary
    for i in range(10):
        try:
            series = await getSeriesPerStudy(study_id, async_client)
            # join with the study_dict we already have
                    #create a list of series dicts (combining metadata from study)
            # merge the dictionaries using ** to unpack the dictionaries (since .union is in place)
            series_list = [] # list to store all the series for a collection

            for s in series:
                merged_dict = {**study_dict, **s}
                series_list.append(merged_dict)        
            return series_list
        except:
            time.sleep(10)
            continue
    
    # if we get here we have a problem that's happened three times
    raise Exception("Failed to get series over set of retries")
    

In [16]:
async def getInstancesPerSeries(series, async_client):
    urlGetInstancesBase = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSOPInstanceUIDs'
    params = {'SeriesInstanceUID': series}
    #async with httpx.AsyncClient() as client:
    r = await async_client.get(urlGetInstancesBase,params=params,timeout=None) #timeout=15.0)
    
    # not really needed here, but should send every time we send a request
    handleRateLimits(r.headers)
    
    if r is not None:
        #return [c['Collection'] for c in r.json()]
        return r.json()

In [17]:
async def getInstancesPerSeriesJoinDict(series_id, series_dict, async_client):
    
    #retry, if necessary
    for i in range(10):
        try:
            instances = await getInstancesPerSeries(series_id, async_client)
            # join with the study_dict we already have
                    #create a list of series dicts (combining metadata from study)
            # merge the dictionaries using ** to unpack the dictionaries (since .union is in place)
            instances_list = [] # list to store all the series for a collection

            for s in instances:
                merged_dict = {**series_dict, **s}
                instances_list.append(merged_dict)   
            return instances_list
        except:
            time.sleep(i)
            continue
    
    # if we get here we have a problem that's happened three times
    raise Exception("Failed to get series over set of retries")
    

## Pull it all together

In [24]:
# Get all the collections. We'll operate on one at a time

tcia_collections = await getCollectionsAsync()

In [25]:
tcia_collections

[{'Collection': 'TCGA-GBM'},
 {'Collection': 'LIDC-IDRI'},
 {'Collection': 'BREAST-DIAGNOSIS'},
 {'Collection': 'PROSTATE-MRI'},
 {'Collection': 'PROSTATE-DIAGNOSIS'},
 {'Collection': 'NaF PROSTATE'},
 {'Collection': 'CT COLONOGRAPHY'},
 {'Collection': 'REMBRANDT'},
 {'Collection': 'RIDER Breast MRI'},
 {'Collection': 'RIDER Lung CT'},
 {'Collection': 'RIDER NEURO MRI'},
 {'Collection': 'RIDER PHANTOM MRI'},
 {'Collection': 'RIDER PHANTOM PET-CT'},
 {'Collection': 'RIDER Lung PET-CT'},
 {'Collection': 'QIBA CT-1C'},
 {'Collection': 'Phantom FDA'},
 {'Collection': 'TCGA-BRCA'},
 {'Collection': 'QIN-BREAST'},
 {'Collection': 'TCGA-LGG'},
 {'Collection': 'TCGA-KIRC'},
 {'Collection': 'TCGA-LUAD'},
 {'Collection': 'Prostate-3T'},
 {'Collection': 'QIN-HEADNECK'},
 {'Collection': 'TCGA-PRAD'},
 {'Collection': 'NSCLC Radiogenomics'},
 {'Collection': 'Head-Neck Cetuximab'},
 {'Collection': 'TCGA-LIHC'},
 {'Collection': 'TCGA-KIRP'},
 {'Collection': 'TCGA-OV'},
 {'Collection': 'TCGA-HNSC'},
 {'

In [22]:
# For each collection, get the Studies, and then the Series
# Hypothesis: pandas is far more memory efficient than Python dicts so first go get every single
#   Study for all passed in collections and put into a DataFrame and THEN go get Series

already_processed_count = 0
for collection in tcia_collections: # iter through the dictionaries in the list
    
    # To save a LOT of time, don't rerun collections if they've already been run

    if os.path.exists(f'data/{collection["Collection"]}-series.json'):
        print(f'Already processed {collection["Collection"]}. Please check the data folder.')
        already_processed_count +=1 
        continue
        
    print(f'Already processed: {already_processed_count}')    
    
    if collection['Collection'] in ['CBIS-DDSM','VICTRE','HNSCC']: #QIBA CT-1C only 486, others over 7,000, HNSCC is 1200
        print(f'Skipping {collection["Collection"]} because of size')
        continue
    
    #NOTE: GBM-DSC-MRI-DRO has ZERO series????
    
    series_list = [] # list to store all the series for a collection
    counter = 0
    # Get the Studies
    
    #with httpx.AsyncClient() as async_client: # This doesn't work with 0.12.1 of HTTPX, must be explicit
    async_client = httpx.AsyncClient()
    print('getting studies')
    studies = await getPatientStudiesPerCollection(collection['Collection'], async_client)
    print(f'studies in {collection["Collection"]} : {len(studies)}')
    #print(f'{collection["Collection"]}_studies.csv')
    study_df = pd.DataFrame.from_dict(studies)
    study_df.to_csv(f'{collection["Collection"]}_studies.csv')

    # Need to make this MUCH faster
#    for study in studies[:3]:
#        series = await getSeriesPerStudy(study['StudyInstanceUID'], async_client)
#        counter += 1
#        if counter % 50 == 0:
#            print(str(counter))

        #create a list of series dicts (combining metadata from study)
#        for s in series:
#            # merge the dictionaries using ** to unpack the dictionaries (since .union is in place)
#            merged_dict = {**study, **s}
#            series_list.append(merged_dict)

    # Try this instead
    study_ids = [s['StudyInstanceUID'] for s in studies]
    batchsize = 10    


    for i in range(0,len(study_ids),batchsize):
        start = time.time()                  
        trimmed_range = [y for y in range(i,i+batchsize) if y<len(study_ids)]   #only used to shorten the last batch    
        print(trimmed_range)
        #trimmed_study_ids = [study_ids[i] for i in trimmed_range]
        #print(trimmed_study_ids)
        trimmed_studies = [studies[i] for i in trimmed_range]
        #print(trimmed_studies)
        #1.3.6.1.4.1.14519.5.2.1.7695.4001.130563880911723253267280582465
        responses = await asyncio.gather(*[getSeriesPerStudyJoinDict(s["StudyInstanceUID"],s, async_client) for s in trimmed_studies])
        #responses = await asyncio.gather(*[getSeriesPerStudyJoinDict('1.3.6.1.4.1.14519.5.2.1.7695.4001.130563880911723253267280582465',s, async_client) for s in trimmed_studies])
        # unfortunately responses is a list of list of dicts, we need to flatten
        flatten_matrix = [val for sublist in responses for val in sublist] 
        #print(len(flatten_matrix))
        [series_list.append(r) for r in flatten_matrix]
        #print(len(series_list))
        print(f'Studies {i}-{i+batchsize} in {time.time() - start} seconds')  
    #print(len(series_list))    
    
    


    # Output results to a data folder to avoid having to burn time running again
    study_df = pd.DataFrame.from_dict(series_list)
    study_df.to_csv(f'data/{collection["Collection"]}_studies_series.csv')

    # Also save just the resulting list, since that can be useful, too.  :-)
    with open(f'data/{collection["Collection"]}-series.json',"w") as f:
        json.dump(series_list, f)
    
    # close the connection explicitly, until with is supported by HTTPX
    await async_client.aclose()
    
    

    

Already processed: 0
getting studies
studies in TCGA-GBM : 575
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Studies 0-10 in 33.315736055374146 seconds
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Studies 10-20 in 31.111398935317993 seconds
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Studies 20-30 in 31.08149242401123 seconds
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
Studies 30-40 in 30.402886390686035 seconds
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Studies 40-50 in 31.92508578300476 seconds
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
Studies 50-60 in 29.77124834060669 seconds
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
Studies 60-70 in 30.29601740837097 seconds
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
Studies 70-80 in 30.1108078956604 seconds
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
Studies 80-90 in 33.08348560333252 seconds
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Studies 90-100 in 36.555383920669556 seconds
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
Studies 100-110 in 35.85938000679016 seconds
[110, 11

## Handle Series Instances

In [23]:
## use existing TCIA collections
for collection in tcia_collections: # iter through the dictionaries in the list
    
    # Don't try to do items that aready exist
    if not os.path.exists(f'data/{collection["Collection"]}-series.json'):
        print(f'Cannot find csv for {collection["Collection"]}. Please check the data folder.')
        continue
        
    if collection['Collection']=="LDCT-and-Projection-data":
        print('skipping LDCT and Projection data')
        continue
        
    if os.path.exists(f'data/{collection["Collection"]}-instances.json'):
        print(f'Already processed {collection["Collection"]}. Please check the data folder.')
        continue
        
    print(f'Processing {collection["Collection"]}.')
    with open(f'data/{collection["Collection"]}-series.json', 'r') as myfile:
        data = myfile.read()

    data = data.replace("\'", "\"") # stupidly encoded my JSON with single quotes, instead of double - fix that with hack
    series=json.loads(data)
    
    async_client = httpx.AsyncClient()
 
    instance_list = []    
    #series_ids = [s['SeriesInstanceUID'] for s in series]
    batchsize = 50    

    #print(series_ids[:10])   
    #series = series[:60]
    print(f'Processing {len(series)} series.')

    for i in range(0,len(series),batchsize):               
        trimmed_range = [y for y in range(i,i+batchsize) if y<len(series)]   #only used to shorten the last batch    
        #print(f'trimmed_range is {trimmed_range}')

        trimmed_series = [series[i] for i in trimmed_range]
        #print(len(trimmed_series))
        start = time.time()   
        
      
        responses = await asyncio.gather(*[getInstancesPerSeriesJoinDict(s["SeriesInstanceUID"],s, async_client) for s in trimmed_series])
        print(f'Series {i}-{i+batchsize} in {time.time() - start} seconds')
        #print(len(responses))
        # unfortunately responses is a list of list of dicts, we need to flatten
        flatten_matrix = [val for sublist in responses for val in sublist] 
        #print(len(flatten_matrix))
        [instance_list.append(r) for r in flatten_matrix]
    
    print(len(instance_list))
    


    # Output results to a data folder to avoid having to burn time running again
    df = pd.DataFrame.from_dict(instance_list)
    df.to_csv(f'data/{collection["Collection"]}_studies_series_instances.csv')

    # Also save just the resulting list, since that can be useful, too.  :-)
    with open(f'data/{collection["Collection"]}-instances.json',"w") as f:
        json.dump(instance_list, f)
    
    # close the connection explicitly
    await async_client.aclose()
    
    

Processing TCGA-GBM.
Processing 6208 series.
Series 0-50 in 7.066700220108032 seconds
Series 50-100 in 8.26707649230957 seconds
Series 100-150 in 3.630408525466919 seconds
Series 150-200 in 1.256850242614746 seconds
Series 200-250 in 1.352020263671875 seconds
Series 250-300 in 1.8067431449890137 seconds
Series 300-350 in 1.7508928775787354 seconds
Series 350-400 in 1.5195119380950928 seconds
Series 400-450 in 1.614271640777588 seconds
Series 450-500 in 1.318326473236084 seconds
Series 500-550 in 1.5159130096435547 seconds
Series 550-600 in 1.4819550514221191 seconds
Series 600-650 in 2.27018141746521 seconds
Series 650-700 in 2.9213478565216064 seconds
Series 700-750 in 1.8629369735717773 seconds
Series 750-800 in 2.276095151901245 seconds
Series 800-850 in 2.502464532852173 seconds
Series 850-900 in 1.7380445003509521 seconds
Series 900-950 in 2.0274152755737305 seconds
Series 950-1000 in 2.1989922523498535 seconds
Series 1000-1050 in 2.890496015548706 seconds
Series 1050-1100 in 2.32

In [ ]:
len(instance_list)

In [ ]:
len(series)

In [ ]:
print(len(study_df))
study_df.head()

In [ ]:
for x in series_list[:1]:
    print(x)

In [ ]:
series_list2 = list(set(series_list))

In [ ]:
# delete
study_ids = study_ids[0:2]

for i in range(0,len(study_ids),batchsize):
    start = time.time()                  
    trimmed_range = [y for y in range(i,i+batchsize) if y<len(study_ids)]   #only used to shorten the last batch    
    print(trimmed_range)
    trimmed_study_ids = [study_ids[sid] for sid in trimmed_range]
    print(trimmed_study_ids)

    responses = await asyncio.gather(*[getSeriesPerStudyJoinDict(sid, async_client) for sid in trimmed_study_ids])
    # unfortunately responses is a list of list of dicts, we need to flatten
    flatten_matrix = [val for sublist in responses for val in sublist] 
    #print(len(flatten_matrix))
    [series_list.append(r) for r in flatten_matrix]
    print(len(series_list))
    print(f'Studies {i}-{i+batchsize} in {time.time() - start} seconds')  
#print(len(series_list))    

In [ ]:
    start = time.time()
    await asyncio.gather(*[foo() for x in range(i)])
    print(f'{i} runs in {time.time() - start} seconds')

# Throttle requests to 50 concurrent
semaphore = asyncio.Semaphore(50)
#...
async def limit_wrap(url):
    async with semaphore:
        # do what you want
#...
results = asyncio.gather([limit_wrap(url) for url in urls])


In [ ]:
study_ids = [s['StudyInstanceUID'] for s in studies]
stuff = []

l = [x for x in range(len(study_ids))]
batchsize = 20
for x in range(0,len(l),batchsize):
    #print('--------')
    trimmed = [y for y in range(x,x+batchsize) if y<len(l)]
    print(trimmed)

    for a in l[x:x+len(trimmed)]:
        stuff.append(a)
        
print(len(stuff))
print(len(study_ids))
    

In [ ]:
import time
study_ids = [s['StudyInstanceUID'] for s in studies]
print(len(study_ids))


series_list = [] # list to store all the series for a collection

entire_start = time.time()
async_client = httpx.AsyncClient()
batchsize = 10
batches=2
for i in range(0,batchsize*batches,batchsize):
    start = time.time()
    responses = await asyncio.gather(*[getSeriesPerStudy("1.3.6.1.4.1.14519.5.2.1.7695.4001.130563880911723253267280582465", async_client) for sid in study_ids[i:i+batchsize]])
    # unfortunately responses is a list of list of dicts, we need to flatten
    flatten_matrix = [val for sublist in responses for val in sublist] 
    print(len(flatten_matrix))
    [series_list.append(r) for r in flatten_matrix]
    print(len(series_list))
    print(f'Studies {i}-{i+batchsize} in {time.time() - start} seconds')  
await async_client.aclose()
print(time.time() - entire_start)
print(len(series_list))
# 2 batches of 20 studies is 98 seconds, series len = 160
# 2 batches of 10 studies is 50 seconds, series len = 80
# 2 batches of  5 studius is 26 seconds, series len = 40
# 2 batches of  2 studies is 22 seconds, series len = 16
# 2 batches of  1 study   is 15 seconds, series len = 8
# kind of linear growth

In [ ]:
print(len(responses))
print(len(series_list))

In [ ]:
flatten_matrix = [val for sublist in series_list for val in sublist] 
print(len(flatten_matrix))

series_list[:2]
df = pd.DataFrame.from_dict(series_list)
print(len(df))
df.head()


In [ ]:
# Need to make this MUCH faster



for study in studies[:3]:
    series = await getSeriesPerStudy(study['StudyInstanceUID'])

    #create a list of series dicts (combining metadata from study)
    for s in series:
        # merge the dictionaries using ** to unpack the dictionaries (since .union is in place)
        merged_dict = {**study, **s}
        series_list.append(merged_dict)

In [ ]:
import json
collection = tcia_collections[1]
with open(f'data/{collection["Collection"]}-series-test.json',"w") as f:
    json.dump(series_list, f)
    #series_list = f.read()

In [ ]:
type(series_list)

In [ ]:
# Load the list if we need to
collection = tcia_collections[1]
with open(f'data/{collection["Collection"]}-series-test.json',"r") as f:
    l = json.load(f)
type(l) # should read 'list'

In [ ]:
q = asq.QueueClient.from_connection_string(conn_str='DefaultEndpointsProtocol=https;AccountName=sjbfunctest;AccountKey=XuYBliYrXazCmfDdK2jLcaJcfqPgu8tC43TlltTMY413nusjx2N6+IvErYmVXuZfOBVgVaCQ52RObKioS9FDRg==;EndpointSuffix=core.windows.net', queue_name='foofoo3')

try:
    p = q.get_queue_properties()
except:
    q.create_queue()
q.send_message('Hello-There ')
r = asq.TextBase64EncodePolicy()
r.encode('TEST-THIS')

In [ ]:
# Works with the addition of async with 

import httpx
import asyncio
import aiofiles

async def download(url:str):
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollections"
    async with httpx.AsyncClient() as client:
        resp = await client.get(url)
    return resp

async def download_lots(i):
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollections"
    await asyncio.gather(*[download(url) for x in range(i)])

#if __name__ == "__main__":
#    asyncio.run(download_lots))  # used outside of Jupyter when I don't have an event loop

for i in range(7):
    start = time.time()
    await download_lots(i)
    print(f'{i} runs in {time.time() - start} seconds')

print('done')

In [ ]:
import httpx
import asyncio
import aiofiles

import os

async def download(url:str, folder:str):
    filename = url.split("/")[-1]
    resp = await httpx.get(url)
    resp.raise_for_status()
    #async with aiofiles.open(os.path.join(folder, filename), "wb") as f:
    #    await f.write(resp.content)


async def download_all_photos(loops: str):
    #resp = httpx.get("https://jsonplaceholder.typicode.com/photos")
    #resp.raise_for_status()
    #urls = list(set(d["url"] for d in resp.json()))[:10]
    #os.makedirs(out_dir, exist_ok=True)
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollectionValues"
    await asyncio.gather(*[download(url, "bob") for x in range(loops)])


#if __name__ == "__main__":
#    asyncio.run(download_all_photos('100_photos'))

for i in range(5):
    start = time.time()
    await download_all_photos(i)
    print(f'{i} runs in {time.time() - start} seconds')

In [ ]:
async with httpx.AsyncClient() as client:
    r = await client.get('https://www.example.com/')
r

In [ ]:
##  This works with HTTPX
async def foo():
    async with httpx.AsyncClient() as client:
        url = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollectionValues'
        r = await client.get(url) #'https://www.example.com/')
    return r


for i in range(5):
    start = time.time()
    await asyncio.gather(*[foo() for x in range(i)])
    print(f'{i} runs in {time.time() - start} seconds')


In [ ]:
import aiohttp
from aiohttp import ClientSession
import asyncio

async def call_url(x, session):
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries?Collection=TCGA-GBM&StudyInstanceUID=1.3.6.1.4.1.14519.5.2.1.7695.4001.130563880911723253267280582465"
    
    response = await session.get(url, timeout=None)
    response_json = await response.json()
    return response_json


async def run_program(x, session):
    """Wrapper for running program in an asynchronous manner"""
    #try:
    response = await call_url(x, session)
        #print(f"Response: {json.dumps(response, indent=2)}")
    #except Exception as err:
        #print(f"Exception occured: {err}")
        #pass



In [ ]:
import time
for i in range(5):
    start = time.time()
    #async with httpx.AsyncClient() as session:
    async with ClientSession as sesssion:
        await asyncio.gather(*[run_program(x,session) for x in range(i)])
   # print(f'{i} runs in {time.time() - start}')

In [ ]:
    try:
        response = await session.request(method='GET', url=url)
        response.raise_for_status()
        print(f"Response status ({url}): {response.status}")
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error ocurred: {err}")

In [ ]:
# Works with aiohttp but not httpx 
import aiohttp
import asyncio
import time
import httpx

async def call_url(session):
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollections"
    #response = await session.request(method='GET', url=url)
    response = await session.get(url=url)

    return response

for i in range(1,5):
    start = time.time() # start time for timing event
    async with aiohttp.ClientSession() as session: #use aiohttp
    #async with httpx.AsyncClient as session:  #use httpx
        await asyncio.gather(*[call_url(session) for x in range(i)])
    print(f'{i} call(s) in {time.time() - start} seconds')

In [ ]:
import aiohttp
import asyncio
import time
import httpx

async def call_url(session):
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollectionValues"
    #response = await session.request(method='GET', url=url)
    response = await session.get(url=url)

    return response

for i in range(1,5):
    start = time.time() # start time for timing event
    #async with aiohttp.ClientSession() as session: #use aiohttp
    session = httpx.AsyncClient() #use httpx
    await asyncio.gather(*[call_url(session) for x in range(i)])
    await session.aclose()
    print(f'{i} call(s) in {time.time() - start} seconds')

In [ ]:
for i in range(1,5):
    start = time.time()
    async with aiohttp.ClientSession() as session:
    #async with httpx.AsyncClient as session:
        await asyncio.gather(*[call_url(session) for x in range(i)])
    print(f'{i} call(s) in {time.time() - start} seconds')

In [ ]:
import aiohttp
import asyncio
import time
import httpx

async def call_url(session):
    url = "https://services.cancerimagingarchive.net/services/v3/TCIA/query/getCollections"
    #async with aiohttp.ClientSession() as session: #use aiohttp
    async with httpx.AsyncClient as session:  #use httpx
        response = await session.get(url=url)

    return response

for i in range(1,5):
    start = time.time() # start time for timing event
    await asyncio.gather(*[call_url(session) for x in range(i)])
    print(f'{i} call(s) in {time.time() - start} seconds')

In [ ]:
print(len(series_list))
print(series_list[:1])



In [ ]:
study_df = pd.DataFrame.from_dict(series_list)
study_df.to_csv(f'data/{tcia_collections[0]["Collection"]}_studies_series.csv')


In [ ]:
len(study_df)
with open('data/TCGA-GBM-series.json',"w") as f:
    f.write(str(series_list))

In [ ]:
series_sample = series[0]    
study_sample = studies[0]    

In [ ]:
 
series_fields = [x for x in series_sample]
study_fields = [x for x in study_sample]
print(len(series_fields))
print(len(study_fields))
merged = list(set(series_fields).union(set(study_fields)))

merged2 = {**study_sample, **series_sample}

print(study_sample)

In [ ]:
study_sample.update(series_sample)
print(len(study_sample))
print(study_sample)

merged2 = {**study_sample, **series_sample}
print(len(merged2))
print(merged2)

In [ ]:
study_sample.update(series_sample)
print(len(study_sample))
study_sample
from collections import OrderedDict
od = OrderedDict(study_sample)
od

x = []
x.append(study_sample)

df = pd.DataFrame.from_dict(x)

df.head()

In [ ]:
print(series_sample)
print(study_sample)


In [ ]:
df = pd.DataFrame.from_dict(studies)

In [ ]:
print(len(df))

In [ ]:
df.head()

In [ ]:
#https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries?Collection=TCGA-GBM&StudyInstanceUID=1.3.6.1.4.1.14519.5.2.1.7695.4001.130563880911723253267280582465

In [ ]:
studies = [s['StudyInstanceUID'] for s in res.json()]
#for x in res.json():
#    print(x['StudyInstanceUID'])
studies

In [ ]:
    for x in collections.json()[:2]:
        print(x['Collection'])

In [ ]:
# Create a list with all the studies
studies = [s['StudyInstanceUID'] for s in res.json()]


#storageConnString = os.environ["AzureWebJobsStorage"]
storageConnString = 'DefaultEndpointsProtocol=https;AccountName=sjbfunctest;AccountKey=XuYBliYrXazCmfDdK2jLcaJcfqPgu8tC43TlltTMY413nusjx2N6+IvErYmVXuZfOBVgVaCQ52RObKioS9FDRg==;EndpointSuffix=core.windows.net'

#x = asq.QueueService(account_name='sjbfunctest', account_key='mykey')
#service = asq.QueueServiceClient.from_connection_string(conn_str=connection_string)
patient_studies_queue = asq.QueueClient.from_connection_string(conn_str=storageConnString,queue_name='studies')

# Create the queue if it doesn't exist...  by exception
#   Which is hacky, but effective
try:
    patient_studies_queue.get_queue_properties()
except:
    patient_studies_queue.create_queue()

# Must base-64 encode since... functions...
enc = asq.TextBase64EncodePolicy()


In [ ]:

for study in studies:
    b64 = enc.encode(study)
    patient_studies_queue.send_message(b64)

In [ ]:
len(studies)
study_id = studies[0]
study_id

In [ ]:
study_id = studies[0]

urlGetSeries = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getSeries'
params = {'StudyInstanceUID': study_id}
res = requests.get(urlGetSeries,params=params,timeout=None) #timeout=15.0)



In [ ]:
res.json()[0]

In [ ]:
# Create a list with all the studies
series = [s['SeriesInstanceUID'] for s in res.json()]

series



In [ ]:
#storageConnString = os.environ["AzureWebJobsStorage"]
storageConnString = 'DefaultEndpointsProtocol=https;AccountName=sjbfunctest;AccountKey=XuYBliYrXazCmfDdK2jLcaJcfqPgu8tC43TlltTMY413nusjx2N6+IvErYmVXuZfOBVgVaCQ52RObKioS9FDRg==;EndpointSuffix=core.windows.net'

series_queue = asq.QueueClient.from_connection_string(conn_str=storageConnString,queue_name='series')

# Create the queue if it doesn't exist...  by exception
#   Which is hacky, but effective
try:
    series_queue.get_queue_properties()
except:
    series_queue.create_queue()

# Must base-64 encode since... functions...
enc = asq.TextBase64EncodePolicy()

for s in series[:1]:
    b64 = enc.encode(s)
    series_queue.send_message(b64)
    print(s)

In [ ]:
# Get the zip files
url = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getImage?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.7695.4001.306204232344341694648035234440'
res = requests.get(url,timeout=None)

In [ ]:
res

In [ ]:
#Download files


In [ ]:
   # Get the study id from the base-64 encoded incoming queue
#series_id = msg.get_body().decode('utf-8')
series_id = series[0]

series_id

In [ ]:
urlGetImage = 'https://services.cancerimagingarchive.net/services/v3/TCIA/query/getImage'
params = {'SeriesInstanceUID': series_id}
res = requests.get(urlGetImage,params=params,timeout=None) #timeout=15.0)
print(res.url)

In [ ]:
import requests
import shutil

def download_file(url):
    local_filename = "foo4.zip"
    with requests.get(urlGetImage, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [ ]:
local_filename = "food4.zip"
with requests.get(urlGetImage,params=params,timeout=None, stream=True) as r:
    with open(local_filename, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [ ]:
import zipfile
import io
from io import BytesIO

file_like_object = io.BytesIO(res.content)
z = zipfile.ZipFile(file_like_object)

In [ ]:
files = z.filelist
f1 = files[1]

#for f in files:
    #print(f)
    #z.read(f)
dcmbytes = z.read(f)
#dcmbytes

In [ ]:
#!pip install azure.storage.blob

import azure.storage.blob as blob

In [ ]:
parts = f1.filename.split('/')
dcm_names = [p for p in parts if p.find('.dcm') != -1]
if len(dcm_names) > 0:
    dcm_name = dcm_names[0]
dcm_name

In [ ]:
storageConnString = 'DefaultEndpointsProtocol=https;AccountName=sjbfunctest;AccountKey=XuYBliYrXazCmfDdK2jLcaJcfqPgu8tC43TlltTMY413nusjx2N6+IvErYmVXuZfOBVgVaCQ52RObKioS9FDRg==;EndpointSuffix=core.windows.net'

b = blob.ContainerClient.from_connection_string(conn_str=storageConnString,container_name='dicoms2')

# Create the queue if it doesn't exist...  by exception
#   Which is hacky, but effective
try:
    b.get_container_properties()
except:
    b.create_container()

for f in files:
    dicom_file = z.read(f)
    parts = f.filename.split('/')
    dcm_parts = [p for p in parts if p.find('.dcm') != -1]
    if len(dcm_parts) == 1: # we have a dicom file, and only one
        dcm_name = f'{series_id}/{dcm_parts[0]}'
        print(dcm_name)
        up = b.upload_blob(data=z.read(f), name=dcm_name)
    
    
    

#upblob = b.upload_blob(data=dcmbytes,name='test3.dcm')

In [ ]:
upblob.blob_name

In [ ]:

storageConnString = 'DefaultEndpointsProtocol=https;AccountName=sjbfunctest;AccountKey=XuYBliYrXazCmfDdK2jLcaJcfqPgu8tC43TlltTMY413nusjx2N6+IvErYmVXuZfOBVgVaCQ52RObKioS9FDRg==;EndpointSuffix=core.windows.net'

series_queue = asq.QueueClient.from_connection_string(conn_str=storageConnString,queue_name='series')

# Create the queue if it doesn't exist...  by exception
#   Which is hacky, but effective
try:
    series_queue.get_queue_properties()
except:
    series_queue.create_queue()

# Must base-64 encode since... functions...
enc = asq.TextBase64EncodePolicy()


In [ ]:



    
    
    
    
    # Create a list with all the studies
    series = [s['SeriesInstanceUID'] for s in res.json()]


    storageConnString = os.environ["AzureWebJobsStorage"]
  
    series_queue = asq.QueueClient.from_connection_string(conn_str=storageConnString,queue_name='series')

    # Create the queue if it doesn't exist...  by exception
    #   Which is hacky, but effective
    try:
        series_queue.get_queue_properties()
    except:
        series_queue.create_queue()

    # Must base-64 encode since... functions...
    enc = asq.TextBase64EncodePolicy()

    for s in series:
        b64 = enc.encode(s)
        series_queue.send_message(b64)

In [ ]:
import httpx
import asyncio
import aiofiles

import os

async def download(url:str, folder:str):
    filename = url.split("/")[-1]
    resp = await httpx.get(url)
    resp.raise_for_status()
    async with aiofiles.open(os.path.join(folder, filename), "wb") as f:
        await f.write(resp.content)


async def download_all_photos(out_dir: str):
    resp = await httpx.get("https://jsonplaceholder.typicode.com/photos")
    resp.raise_for_status()
    urls = list(set(d["url"] for d in resp.json()))[:100]
    os.makedirs(out_dir, exist_ok=True)
    await asyncio.gather(*[download(url, out_dir) for url in urls])


if __name__ == "__main__":
    asyncio.run(download_all_photos('100_photos'))

In [ ]:
## WORKS

import httpx
import asyncio
import aiofiles

import os

async def download(url:str, folder:str):
    filename = url.split("/")[-1]
    async with httpx.AsyncClient() as session:
        resp = await session.get(url)
        resp.raise_for_status()
    async with aiofiles.open(os.path.join(folder, filename), "wb") as f:
        await f.write(resp.content)
        
async def download_all_photos(out_dir: str):
    async with httpx.AsyncClient() as session:
        resp = await session.get("https://jsonplaceholder.typicode.com/photos")
        resp.raise_for_status()
    urls = list(set(d["url"] for d in resp.json()))[:100]
    os.makedirs(out_dir, exist_ok=True)
    await asyncio.gather(*[download(url, out_dir) for url in urls])


#if __name__ == "__main__":
#    asyncio.run(download_all_photos('100_photos'))
    
await download_all_photos('100_photos')

In [ ]:
def write_stuff(f):
    f.write('And stuff with context passed to another method. ')

In [ ]:
def write_stuff(f):
    f.write('And stuff with context passed to another method. ')
    
with open('foo.txt',"w") as f:
    f.write('Start with context manager inside with statement. ')
    write_stuff(f)
    f.write('And back to close the with.')


In [ ]:
!pip show httpx
